In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "F:\ML\Machine learning\Hands-on machine learning with scikit-learn and tensorflow"
PROJECT_ROOT_DIR =  os.getcwd()
CHAPTER_ID = "11_Training Deep Neural Nets"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


In [28]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
#这个地方没明白
he_init = tf.variance_scaling_initializer()
activation=tf.nn.elu
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=activation, name="hidden1",kernel_initializer= he_init )
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=activation, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=activation, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=activation, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=activation, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")
# def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
#         activation=tf.nn.elu, initializer=he_init):
#     with tf.variable_scope(name, "dnn"):
#         for layer in range(n_hidden_layers):
#             inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
#                                      kernel_initializer=initializer,
#                                      name="hidden%d" % (layer + 1))
#         return inputs
# dnn_outputs = dnn(X)
# logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer =  tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32),name = 'accuracy')

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 1000
batch_size = 20

In [29]:
# 下载数据：
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]


X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]


In [30]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
best_loss = np.infty
max_continue_steps = 20
check_continue_steps =0
best_epoch =np.infty
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train1, y_train1, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        #early stop
        loss_val,acc_val = sess.run([loss,accuracy],feed_dict={X:X_valid1,y:y_valid1})
        if loss_val< best_loss:
            best_loss = loss_val
            best_epoch = epoch
            check_continue_steps =0
            saver.save(sess,"./ch11-exercise_model_0_to_4.ckpt")
            
        else :
            check_continue_steps+=1
            if check_continue_steps >= max_continue_steps:
                print("early stop")
                break 
                
        accuracy_val = accuracy.eval(feed_dict={X: X_valid1, y: y_valid1})
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))



0	Validation loss: 0.158328	Best loss: 0.158328	Accuracy: 96.91%
1	Validation loss: 0.115867	Best loss: 0.115867	Accuracy: 97.58%
2	Validation loss: 0.079341	Best loss: 0.079341	Accuracy: 98.24%
3	Validation loss: 0.067289	Best loss: 0.067289	Accuracy: 98.16%
4	Validation loss: 0.568616	Best loss: 0.067289	Accuracy: 81.86%
5	Validation loss: 0.115313	Best loss: 0.067289	Accuracy: 96.91%
6	Validation loss: 0.287698	Best loss: 0.067289	Accuracy: 92.06%
7	Validation loss: 0.308450	Best loss: 0.067289	Accuracy: 94.80%
8	Validation loss: 0.420121	Best loss: 0.067289	Accuracy: 96.60%
9	Validation loss: 0.299701	Best loss: 0.067289	Accuracy: 97.22%
10	Validation loss: 0.245446	Best loss: 0.067289	Accuracy: 96.52%
11	Validation loss: 1.593604	Best loss: 0.067289	Accuracy: 62.04%
12	Validation loss: 0.340582	Best loss: 0.067289	Accuracy: 94.64%
13	Validation loss: 1.179280	Best loss: 0.067289	Accuracy: 62.55%
14	Validation loss: 0.233367	Best loss: 0.067289	Accuracy: 96.29%
15	Validation loss: 

In [31]:
with tf.Session() as sess:
    saver.restore(sess, "./ch11-exercise_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}% best_epoch:{:}".format(acc_test * 100 ,best_epoch))

INFO:tensorflow:Restoring parameters from ./ch11-exercise_model_0_to_4.ckpt
Final test accuracy: 98.40% best_epoch:3
